In [1]:
from __future__ import print_function

import math
import numpy as np
import time
from sklearn import linear_model


def soft_thresholding(x, threshold):
  if x >= threshold:
    return x - threshold
  elif x <= -threshold:
    return x + threshold
  else:
    return 0.0


def compute_objective_value(y, A, x, lambd):
  return 0.5 * np.linalg.norm(y - np.dot(A, x), ord=2)**2 + lambd * np.linalg.norm(x, ord=1)


def compute_x_error(xhat, x):
  return np.linalg.norm(xhat - x, ord=2) / np.linalg.norm(x, ord=2)


def meta_lasso(y, A, lambd, num_effective_passes, order_generator, verbose=False):
  inv_sq_col_norms = np.divide(1.0, np.square(np.linalg.norm(A, axis=0)))
  m, n = A.shape
  x = np.zeros(n)
  r = y
  
  obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
  if verbose:
    print("  Initial objective value: {}".format(obj_value))
  
  stats = [(0.0, obj_value)]
  
  completed_effective_passes = 0.0
  last_print_pass = 0.0
  last_frac_coords= 0.0
  while completed_effective_passes < num_effective_passes:
    gen_verbose = False
    if verbose and completed_effective_passes - last_print_pass + last_frac_coords >= 1.0:
      gen_verbose = True
    coords = order_generator(y, A, x, r, gen_verbose)
    last_frac_coords = len(coords) / float(n)

    num_updated = 0
    prevx = np.copy(x)
    for ii in coords:
      xiold = x[ii]
      thresh = lambd * inv_sq_col_norms[ii]
      val = np.dot(A[:,ii], r) * inv_sq_col_norms[ii] + xiold
      xinew = soft_thresholding(val, thresh)
      if abs(xinew - xiold) > 1e-5:
        num_updated += 1
      r = r + A[:,ii] * (xiold - xinew)
      x[ii] = xinew
    print("    Fraction updated: {}".format(num_updated / float(len(coords))))
    print("    Step size: {}".format(np.linalg.norm(x - prevx)))

    completed_effective_passes += len(coords) / float(n)
    obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
    stats.append((completed_effective_passes, obj_value))
    if verbose and completed_effective_passes - last_print_pass >= 1.0:
      last_print_pass = completed_effective_passes
      print("  Objective value after {} effective passes: {}".format(completed_effective_passes, obj_value))

  return (x, stats)


def lasso_cyclic(y, A, lambd, num_passes, verbose=False):
  _, n = A.shape
  def cyclic_order_generator(*unused):
    return range(n)
  if verbose:
    print("Cyclic coordinate descent Lasso solver")
  return meta_lasso(y, A, lambd, num_passes, cyclic_order_generator, verbose)


def lasso_randomiid(y, A, lambd, num_passes, verbose=False):
  _, n = A.shape
  def randiid_order_generator(*unused):
    return np.random.randint(n, size=n)
  if verbose:
    print("Random iid coordinate descent Lasso solver")
  return meta_lasso(y, A, lambd, num_passes, randiid_order_generator, verbose)


def lasso_randomperm(y, A, lambd, num_passes, verbose=False):
  _, n = A.shape
  def randomperm_order_generator(*unused):
    return np.random.permutation(n)
  if verbose:
    print("Random permutation coordinate descent Lasso solver")
  return meta_lasso(y, A, lambd, num_passes, randomperm_order_generator, verbose)

def lasso_maxip(y, A, lambd, num_passes, prefix_length=-1, verbose=False):
  _, n = A.shape
  if prefix_length < 0:
    prefix_length = n
  def maxip_order_generator(y, A, x, r, gen_verbose):
    ips = np.dot(A.transpose(), r)
    indices = np.flipud(np.argsort(np.abs(ips)))
    if gen_verbose:
      top10prefix = indices[:10]
      top10ips = ips[top10prefix]
      print('    Top 10 indices: {}'.format(top10prefix))
      print('    Top 10 inner products: {} ...'.format(top10ips[:5]))
      print('        ... {}'.format(top10ips[5:]))
      top10ipsnorm = top10ips / np.linalg.norm(r, ord=2)
      top10ipsnorm /= np.linalg.norm(A[:,top10prefix], axis=0)
      print('    Top 10 inner products (normalized): {} ...'.format(top10ipsnorm[:5]))
      print('        ... {}'.format(top10ipsnorm[5:]))
    return indices[:prefix_length]
  if verbose:
    print("Maximum inner product coordinate descent Lasso solver")
  return meta_lasso(y, A, lambd, num_passes, maxip_order_generator, verbose)

def lasso_maxiprandom(y, A, lambd, num_passes, prefix_length=-1, verbose=False):
  _, n = A.shape
  if prefix_length < 0:
    prefix_length = n
  def maxiprandom_order_generator(y, A, x, r, gen_verbose):
    if np.random.randint(2) == 0:
      print('    Random permutation')
      indices = np.random.permutation(n)
    else:
      print('    Maxip permutation')
      ips = np.dot(A.transpose(), r)
      indices = np.flipud(np.argsort(np.abs(ips)))
    if gen_verbose:
      top10prefix = indices[:10]
      top10cols = A[:,top10prefix]
      top10ips = np.dot(top10cols.transpose(), r)
      print('    Top 10 indices: {}'.format(top10prefix))
      print('    Top 10 inner products: {} ...'.format(top10ips[:5]))
      print('        ... {}'.format(top10ips[5:]))
      top10ipsnorm = top10ips / np.linalg.norm(r, ord=2)
      top10ipsnorm /= np.linalg.norm(top10cols, axis=0)
      print('    Top 10 inner products (normalized): {} ...'.format(top10ipsnorm[:5]))
      print('        ... {}'.format(top10ipsnorm[5:]))
    return indices[:prefix_length]
  if verbose:
    print("Maximum inner product random coordinate descent Lasso solver")
  return meta_lasso(y, A, lambd, num_passes, maxiprandom_order_generator, verbose)







#def run_lasso_experiment(y, A, lambd, num_passes





###################################################################
# OLD CODE

def lasso_cyclic_explicit(y, A, lambd, num_passes, verbose=False):
  if verbose:
    print("Cyclic coordinate descent Lasso solver")
  inv_sq_col_norms = np.divide(1.0, np.square(np.linalg.norm(A, axis=0)))
  m, n = A.shape
  x = np.zeros(n)
  r = y

  if verbose:
    obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
    print("  Initial objective value: {}".format(obj_value))


  for cur_cycle in range(num_passes):
    num_nonzero = 0
    for ii in range(n):
      xiold = x[ii]
      thresh = lambd * inv_sq_col_norms[ii]
      val = np.dot(A[:,ii], r) * inv_sq_col_norms[ii] + xiold
      xinew = soft_thresholding(val, thresh)
      r = r + A[:,ii] * (xiold - xinew)
      x[ii] = xinew
      if abs(xinew) > 1e-6:
        num_nonzero += 1

    if verbose:
      obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
      print("  Current objective value: {}  ({} nonzero coordinates)".format(obj_value, num_nonzero))
  return x


def lasso_random_explicit(y, A, lambd, num_steps, verbose=False):
  if verbose:
    print("Random coordinate descent Lasso solver")
  inv_sq_col_norms = np.divide(1.0, np.square(np.linalg.norm(A, axis=0)))
  m, n = A.shape
  x = np.zeros(n)
  r = y

  if verbose:
    obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
    print("  Initial objective value: {}".format(obj_value))

  for cur_step in range(num_steps):
    ii = np.random.randint(0, n)
    xiold = x[ii]
    thresh = lambd * inv_sq_col_norms[ii]
    val = np.dot(A[:,ii], r) * inv_sq_col_norms[ii] + xiold
    xinew = soft_thresholding(val, thresh)
    r = r + A[:,ii] * (xiold - xinew)
    x[ii] = xinew
    
    if verbose and cur_step % n == 0 and cur_step > 0:
      obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
      print("  Current objective value: {}".format(obj_value))
  return x


def lasso_steepest(y, A, lambd, num_steps, verbose=False):
  if verbose:
    print("Steepest coordinate descent Lasso solver")
  inv_sq_col_norms = np.divide(1.0, np.square(np.linalg.norm(A, axis=0)))
  m, n = A.shape
  x = np.zeros(n)
  r = y
  r_values = []

  if verbose:
    obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
    print("  Initial objective value: {}".format(obj_value))

  for cur_step in range(num_steps):
    derivatives = np.abs(np.dot(np.transpose(A), r / np.linalg.norm(r, ord=2)))
    r_values.append(r / np.linalg.norm(r, ord=2))
    ii = np.argmax(derivatives)
    # Alternative coordinate selection methods (e.g., largest progress)
    #  #derivatives = np.abs(np.dot(np.transpose(A), r) - lambd * np.sign(x))
    #print("  Largest inner product: {}".format(derivatives[ii]))
    #ii = -1
    #best_abs = 0.0
    #for jj in range(n):
    #  xjold = x[jj]
    #  thresh = lambd * inv_sq_col_norms[jj]
    #  val = np.dot(A[:,jj], r) * inv_sq_col_norms[jj] + xjold
    #  xjnew = soft_thresholding(val, thresh)
    #  if abs(xjold - xjnew) > best_abs:
    #    ii = jj
    #    best_abs = abs(xjold - xjnew)
      
    xiold = x[ii]
    thresh = lambd * inv_sq_col_norms[ii]
    val = np.dot(A[:,ii], r) * inv_sq_col_norms[ii] + xiold
    xinew = soft_thresholding(val, thresh)
    #print("    diff = {}  thresh = {}  xiold = {}  xinew = {}".format(np.dot(A[:,ii], r) * inv_sq_col_norms[ii], thresh, xiold, xinew))
    r = r + A[:,ii] * (xiold - xinew)
    x[ii] = xinew
   
    if verbose: 
      obj_value = 0.5 * np.linalg.norm(r)**2 + lambd * np.linalg.norm(x, ord=1)
      #print("  Actually largest inner product: {} (coord {})".format(np.max(derivatives), np.argmax(derivatives)))
      print("  Largest inner product: {:.3} (coord {})  Average inner product: {:.4}  Current objective value: {}".format(derivatives[ii], ii, np.mean(derivatives), obj_value))
  return x, r_values


# From https://github.com/adarshvjois/LASSO/blob/master/HW2/shooting_algorithm.py
def lasso_cyclic_reference(y, X, lambda_reg, num_iter, w_init=None):
    """
    X - Training set
    y - training values to classify/do regression on.
    w_init - my initial guess for the weights
    lambda_reg - hyper parameter
    
    returns the final weight vector.
    """
    lambda_reg *= 2   # added by Ludwig
    #TODO add a way in which I can include tolerance.
    if w_init is not None:
        w = w_init 
    else:
        w = np.zeros(X.shape[1])
    D = X.shape[1]

    XX2 = np.dot(X.T, X) * 2
    Xy2 = np.dot(X.T, y) * 2
    i = 0
    while i < num_iter:
        i += 1
        for j in range(D):
            c_j = Xy2[j] - XX2[j, :].dot(w) + XX2[j, j] * w[j]
            a_j = XX2[j, j]
            
            if c_j < -lambda_reg:
                w[j] = (c_j + lambda_reg) / a_j
            elif c_j > lambda_reg:
                w[j] = (c_j - lambda_reg) / a_j
            else:
                w[j] = 0
    return w


def lasso_sklearn(y, A, lambd, tol=1e-5, max_iter=100000, selection='random'):
  m, n = A.shape
  lambda_prime = lambd / n

  lasso = linear_model.Lasso(alpha=lambda_prime, copy_X=True, normalize=False, positive=False, tol=tol, max_iter=max_iter, fit_intercept=False, selection=selection)
  lasso.fit(A, y)
  return lasso.coef_


In [2]:
%pylab
%matplotlib inline

import math
import numpy as np
import time
from sklearn import linear_model
# Experiment parameters
np.random.seed(5259783)
n = 1000
m = 200
k = 300

num_iter_cyclic = 100
num_iter_steepest = 50
verbose = True
sklearn_tol = 1e-5
sklearn_maxiter = 100000
sklearn_selection = 'random'

# Generate matrix A
A = np.random.randn(m, n)
# normalize
col_norms = np.linalg.norm(A, axis=0)
A = np.divide(A, col_norms)

# Generate x
x = np.zeros(n)
coords = np.random.choice(n, k, replace=False)
#x[coords] = 1.0 - 2.0 * np.random.randint(0, 2, k)
x[coords] = 1.0

# Generate y
y = np.dot(A, x)
#lambd = 0.01 * math.log(n, 2) / m
lambd = 0.1 * math.log(n, 2) / m
print("lambda = {}".format(lambd))

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
lambda = 0.00498289214233


In [9]:
num_iter = 50
lasso_randomperm(y, A, lambd, num_iter, verbose=True)


Random permutation coordinate descent Lasso solver
  Initial objective value: 148.253772936
    Fraction updated: 0.992
    Step size: 17.0723335755
  Objective value after 1.0 effective passes: 2.52148607868
    Fraction updated: 1.0
    Step size: 1.26409959062
  Objective value after 2.0 effective passes: 1.71272379168
    Fraction updated: 0.988
    Step size: 0.240359878214
  Objective value after 3.0 effective passes: 1.68342759232
    Fraction updated: 0.979
    Step size: 0.197946006555
  Objective value after 4.0 effective passes: 1.66366688271
    Fraction updated: 0.971
    Step size: 0.204207338164
  Objective value after 5.0 effective passes: 1.64250916037
    Fraction updated: 0.969
    Step size: 0.201455057996
  Objective value after 6.0 effective passes: 1.62204487804
    Fraction updated: 0.962
    Step size: 0.190934109322
  Objective value after 7.0 effective passes: 1.60350621181
    Fraction updated: 0.951
    Step size: 0.187885390994
  Objective value after 8.0 

(array([  7.04108339e-01,   0.00000000e+00,   2.54844858e-01,
          0.00000000e+00,   0.00000000e+00,  -2.67811130e-01,
          0.00000000e+00,   3.78858085e-02,   0.00000000e+00,
         -9.98231703e-03,   4.34462842e-01,   0.00000000e+00,
          5.07823187e-01,  -6.84034774e-03,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          7.69627449e-03,   5.08483339e-02,   0.00000000e+00,
         -1.59635789e-02,   0.00000000e+00,  -3.17333429e-01,
         -1.93814679e-02,  -7.26008765e-01,   0.00000000e+00,
          5.19527094e-04,   8.50762405e-02,   6.47995406e-01,
          0.00000000e+00,   1.57515819e-01,  -5.48077747e-01,
         -7.48265856e-01,  -3.29066331e-01,  -3.45636304e-01,
          4.07756930e-01,   8.96238212e-02,   2.61870118e-01,
         -2.89266760e-02,  -1.66769928e-01,   3.72088884e-01,
          1.95961880e-01,  -2.24216476e-04,  -2.17873269e-01,
          5.87487807e-02,  -1.62341423e-01,   4.58799159e-02,
        